# TP DE REGRESSION LINEAIRE

**Crédits.** Ce TP est largement inspiré du scénario disponible sur Wikistat.

# 1. Introduction



### Objectif

L'objectif de cette est d'améliorer la prévision déterministe (MOCAGE), calculée par les services de Météo France, de la concentration d'ozone dans certaines stations de prélèvement. Il s'agit d'un problème dit d'adaptation statistique ou post-traitement d'une prévision locale de modèles à trop grande échelle en s'aidant d'autre variables également gérées par MétéoFrance, mais à plus petite échelle (température, force du vent, etc.).

### Description des données 

Les données ont été extraites et mises en forme par le service concerné de Météo France. Elles sont décrites par les variables suivantes:
- JOUR : Type de jour (férié (1) ou pas (0) ;
- O3obs : Concentration d'ozone effectivement observée le lendemain à 17h locales correspondant souvent au maximum de pollution observée ;
- MOCAGE : Prévision de cette pollution obtenue par un modèle déterministe de mécanique des fluides (équation de Navier et Stockes);
- TEMPE : Température prévue par MétéoFrance pour le lendemain 17h ;
- RMH2O : Rapport d'humidité ;
- NO2 : Concentration en dioxyde d'azote ;
- NO : Concentration en monoxyde d'azote ;
- STATION : Lieu de l'observation (Aix-en-Provence, Rambouillet, Munchhausen, Cadarache et Plan de Cuques) ;
- VentMOD : Force du vent ;
- VentANG : Orientation du vent.

Ce sont des données bien codées et de petite taille. Elles présentent avant tout un caractère pédagogique.

## 2. Prise en main des données

Afin de charger et d'étudier les données, nous allons utiliser la librairie pandas pour bénéficier de la classe DataFrame.

### 2.1. Chargement des données

#### Importation des librairies pour le traitement

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt, log

In [ ]:
# chargement des données
Ozone_DF = pd.read_csv('depSeuil.csv', sep = ',', header = 0)

# typage des données
Ozone_DF["STATION"] = pd.Categorical(Ozone_DF["STATION"], ordered = False)
Ozone_DF["JOUR"] = pd.Categorical(Ozone_DF["JOUR"], ordered = False)
Ozone_DF["O3obs"] = pd.DataFrame(Ozone_DF["O3obs"], dtype = float)

In [ ]:
#visualisation du typage des données
Ozone_DF.dtypes

In [ ]:
#visualisation du jeu de données
Ozone_DF.head()

In [ ]:
#visualisation des details du jeu de données
Ozone_DF.info()

ici nous avons 1041 entrées dans notre jeu de données, deux variables categorisée (JOUR ET STATION) et huit variables type float

### 2.2. Exploration des données

### Exploration uni-dimensionelle

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# histogramme des données
fig, axs = plt.subplots(nrows = 3, ncols = 3)

i_name = 0
            
for i in range(3):
    for j in range(3):
            i_name = i_name+1
            name_col = Ozone_DF.columns[i_name]
            axs[i, j].hist(Ozone_DF[name_col])
            axs[i, j].set_title(name_col)

fig.tight_layout()
plt.show()

### Transformation du jeu de données

In [ ]:
from math import sqrt, log

In [ ]:
Ozone_DF["SRMH2O"] = Ozone_DF["RMH2O"].map(lambda x: sqrt(x))
Ozone_DF["LNO2"] = Ozone_DF["NO2"].map(lambda x: log(x))
Ozone_DF["LNO"] = Ozone_DF["NO"].map(lambda x: log(x))
del Ozone_DF["RMH2O"]
del Ozone_DF["NO2"]
del Ozone_DF["NO"]

### Réponses aux questions

Question 1 : ici nous avons effectué les tranformations suivnant:
        remplacer respectivement les colones RMH2O, NO2, NO, par les colones SRMH2O,LNO2,LNO 
        remplacer chaque valeur d'observation de RMH2O par sa racine carrée et celles de NO,, NO2 par le log
        Objectif:
        Cette transformation nous d'avoir une distribution uniforme des données

In [ ]:
Ozone_Dum = pd.get_dummies(Ozone_DF[["JOUR", "STATION"]])
del Ozone_Dum["JOUR_0"]

Ozone_Quant = Ozone_DF[["MOCAGE", "TEMPE", "VentMOD", "VentANG", "SRMH2O", "LNO2", "LNO"]]

Question 2 : ici nous avons convertis les variables catégorielle("JOUR", "STATION") en des variables fictives/indicateurs.
    et nous avons gardé que les variables quantificateurs dans notre DataFrame
    objectif: Cette transformation nous permettra d'avoir que des valeurs quantitatives continues que l'on pourra utiliser uterieurement pour effectuer des calcules ou des mesures 

### Exploration multi-dimensionnelle

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(Ozone_DF[["O3obs", "MOCAGE", "TEMPE", "VentMOD", "VentANG", "SRMH2O", "LNO2", "LNO"]], alpha=0.2, figsize=(15, 15), diagonal='kde')
plt.show()

### 2.3. Création de l'ensemble d'apprentissage / validation et de l'ensemble de test

In [ ]:
X = pd.concat([Ozone_Dum, Ozone_Quant], axis = 1)
Y = Ozone_DF["O3obs"]

X.head()

In [ ]:
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  

In [ ]:
X_1, X_2, Y_1, Y_2 = train_test_split(X, Y, train_size = 400, test_size = 200, random_state = 42)

X_av = X_1.to_numpy()
X_t = X_2.to_numpy()
Y_av = np.transpose([Y_1.to_numpy()])
Y_t = np.transpose([Y_2.to_numpy()])

**Remarque.** Scikit-learn fonctionne avec des numpy array. Nous allons donc utiliser les indices des colonnes:

0. JOUR
1. STATION_Aix 
2. STATION_Als 
3. STATION_Cad 
4. STATION_Pla
5. STATION_Ram
6. MOCAGE
7. TEMPE
8. VentMOD
9. VentANG
10. SRMH2O
11. LNO2
12. LNO

## 3. Régression linéaire univariée

### 3.1. Relation unidimensionelle 

On étudie la qualité de la relation $\hat{Y} = X$, où $\hat{Y}$ est le vecteur de prédiction de l'O3 observée et $X$ le vecteur des prévisions de l'O3 observée calculée par Météo France.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 2)

axs[0].plot(X_av[:,[6]], Y_av, 'o')
axs[0].plot([0, 300], [0, 300], 'r')
axs[0].set_xlabel('Mocage: $X^6 = \hat{Y}_{av}$')
axs[0].set_ylabel('O3 observée: $Y_{av}$')

axs[1].plot(X_av[:,[6]], Y_av - X_av[:,[6]], 'o')
axs[1].plot([0, 300], [0, 0], 'r')
axs[1].set_xlabel('Mocage: $X^6= \hat{Y}_{av}$')
axs[1].set_ylabel('Résidus')

fig.tight_layout()
plt.show()

print("Coefficient de détermination :", r2_score(Y_av, X_av[:,[6]]))

**Question.**: Commentaire des résultats:

Dans notre cas d'etude ici, nous utilisons une relation unidimensionnel en entrée (où les observations dépendent seulement de la variable x = MOCAGE).

Clairement, d'après la visualisation, on peut se dire que la concentration d'ozone effectivement observée dépend de manière linéaire de la prévision de cette pollution (MOCAGE).On peut donc émettre une hypothèse de modélisation qui est que le phénomène possède la forme d'une droite. 

Aussi, on peut voir que lorsque la previson de pollution (Mocage) devient un peu trop grande, les données semblent devenir plus concentrées c'est à dire moins modélisables facilement, il y a plus de variabilité. 

## 3.2. Régression linéaire ordinaire 

On étudie la qualité de la relation $\hat{Y} = X\beta + \epsilon$, où $\hat{Y}$ est le vecteur de prédiction de l'O3 observée et $X$ le vecteur des prévisions de l'O3 observée calculée par Météo France.

In [ ]:
from sklearn import linear_model

In [ ]:
ols = linear_model.LinearRegression()
ols.fit(X_av[:,[6]], Y_av)
Y_hat_av = ols.predict(X_av[:,[6]])

print("Coefficient de régression : \n", ols.coef_)
print()

fig, axs = plt.subplots(nrows = 1, ncols = 2, sharex = True)

axs[0].plot(X_av[:,[6]], Y_av, 'o')
axs[0].plot(X_av[:,[6]], Y_hat_av, 'r-')
#axs[0].set_ylim(0, 300)
axs[0].set_xlabel('Régression Mocage: $\hat{Y}_{av}$')
axs[0].set_ylabel('O3 observée: $Y_{av}$')

axs[1].plot(Y_hat_av, Y_av - Y_hat_av, 'o')
axs[1].plot([0, 300], [0, 0], 'r')
axs[1].set_xlabel('Prédiction Mocage: $\hat{Y}_{av}$')
axs[1].set_ylabel('Résidus')

fig.tight_layout()
plt.show()

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av))

**Questions.**

Q1. Pourquoi n'y a-t-il qu'un seul coefficient ?

R1 :Nous avons un seul coefficient de regression car nous avons consideré qu'une seule variable (MOCAGE) sur notre jeu de donné

Q2. Commentez les résultats.
ici nous voyons que le coefficient de determination est plus important sur le modele où nous avons appliquer une apprentissage sur notre jeu de données (0.35) alors que dans notre premier nous avons mesurer le coefficient de determination juste en fonction de la linaerité de notre variable considée en fonction du phenomene observé (0.17). Ainsi la valeur (0.35) optenue à la suite de l'apprentissage semble etre plus proches de realité du phenomene observé que celle mesurée (0.17)  

# 4. Régression linéaire multivariée

### 4.1. Régression linéaire ordinaire

**Resolution de l'Exercice.** 
Question 1: Appliquez une régression linéaire sur l'ensemble d'apprentissage $(X_{av}, Y_{av})$ et évaluez la qualité des résultats en comparantc eux obtenus sur $(X_{av}, Y_{av})$ et sur $(X_{t}, Y_{t})$. Commentez.

In [ ]:
ols = linear_model.LinearRegression()
model_lin = ols.fit(X_av, Y_av)
Y_hat_t = ols.predict(X_t)

print("Coefficient de régression : \n", ols.coef_)
print()

print("Erreur quadratique (TestSet) : %.2f" % mean_squared_error(Y_t, Y_hat_t))
print("Coefficient de détermination (TestingSet) : %.2f" % r2_score(Y_t, Y_hat_t))

***commentaire***:
Ici nous appris sur l'ensemble de notre jeu de donnnées(X_av) mais nous avons testé avec notre jeu de données testingSet(X_t) . Nous voyons clairement que l'erreur quadratique obtenue sur l'ensemble de notre jeu de données (959.51) est moins importante que celle obtenue si on considere une seule variable (1125.92)le coefficient de determination est inferieur (0.42) que celui obtenu avec une seule variable (0.35).Donc notre modele a une capacité de generalisation meilleure si l'on considere lensemble des variables de notre jeu de données 

In [ ]:
coef = pd.Series(model_lin.coef_[0], index =  X_1.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients de regression Lineaire")
plt.show()

***Question 2***: Observez les coefficients de régression et commentez.

ici nous avons 13 coeficients de regressions qui varient selon les variables observés et on remarque que la SRMH2O impact le plus sur le modele

## 4.2. Régression linéaire pénalisée : Ridge

## Réponses aux questions

Q1: Le but principal de la régression Ridge est de trouver les coefficients qui minimisent la somme des carrés d'erreur en appliquant une pénalité à ces coefficients.

Ce paramètre de réglage est déterminé comme alpha dans le modèle. Tout d'abord, nous configurons le modèle avant de trouver le paramètre de réglage optimal.

Les coefficients du modèle de régression établi peuvent être vus comme suit.

La constante du modèle peut être vue comme suit.

Créons un ensemble aléatoire de valeurs alpha pour trouver le paramètre alpha optimal.

Sauvegardons l'ensemble sous lambda_values.

Ensuite, nous construisons un modèle Ridge. De plus, nous créons un ensemble vide de coefficients. Nous créons le modèle en ajustant chaque valeur alpha dans l'ensemble de valeurs alpha que nous avons créé, puis en ajoutant les coefficients calculés à l'ensemble de coefficients que nous avons créé précédemment.

Nous pouvons voir comment les coefficients changent en fonction des données alpha

Nous allons observer sur l'ensemble d'apprentissage, pour 3 valeurs de alpha ( 𝜆  dans le cours), le comportement de la regression ridge sur les coefficients.



#### Modèle 1

In [ ]:
r_a01 = linear_model.Ridge(alpha = 0.1)
model_r_a01 = r_a01.fit(X_av, Y_av)
Y_hat_av_r01 = r_a01.predict(X_av)
Y_hat_t_r01 = r_a01.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r01))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r01))

coef = pd.Series(model_r_a01.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 0.1")
plt.show()

#### Modèle 2

In [ ]:
r_a1 = linear_model.Ridge(alpha = 1)
model_r_a1 = r_a1.fit(X_av, Y_av)
Y_hat_av_r1 = r_a1.predict(X_av)
Y_hat_t_r1 = r_a1.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r1))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r1))

coef = pd.Series(model_r_a1.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 1")
plt.show()

#### Modèle 3

In [ ]:
r_a10 = linear_model.Ridge(alpha = 10)
model_r_a10 = r_a10.fit(X_av, Y_av)
Y_hat_av_r10 = r_a10.predict(X_av)
Y_hat_t_r10 = r_a10.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r10))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r10))

coef = pd.Series(model_r_a10.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 10")
plt.show()

# Q1. Commentaitre : 
Apres visualisation, nous constatons que l'évolution du comportement sur les coefficients evoluent selon que la valeur de alpha augmente. Cela s'explique du fait que plus alpha est grand plus on penalise les coefficients et plus alpha est proche de 0 moins on penalise et plus le fait de minimiser l'erreur de prediction contraint 

#### Solution de l'exercice :  Visualisons globalement l'évolution des coefficients en fonction de alpha.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Ridge

alphas = [0.1, 1, 10]
coefs = []
for a in alphas:
    ridge = linear_model.Ridge(alpha=a, fit_intercept=False)
    ridge.fit(X_av, Y_av)
    coefs.append(ridge.coef_[0])
    
ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale("log")
#ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel("alpha")
plt.ylabel("Coefs")
plt.title("Ridge coefficients as a function of the regularization")
plt.axis("tight")
plt.show()

## Mettez en place une procédure d'apprentissage rigoureuse pour trouver le paramètre alpha optimal de la régression ridge.

### Etape 1 : Application d'une regression lineaire simple

## Application de la régression ridge

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#Instanciation du model
ridge_reg = linear_model.Ridge()

#definition des valeur de alpha
params_Ridge = {'alpha': [0.1, 1, 10] , "fit_intercept": [True, False]}


#Recherche de meilleur parametre
Ridge_GS = GridSearchCV(ridge_reg,
                        param_grid=params_Ridge,
                       )
#Apprentissage du model
ridge = Ridge_GS.fit(X_av, Y_av)

scores = Ridge_GS.cv_results_["mean_test_score"]
scores_std = Ridge_GS.cv_results_["std_test_score"]

print("Résultats de la validation Croisé sur l'ensemble des paramètres:")   
print("Meilleurs paramètres alpha du Ridge:")
print(Ridge_GS.best_params_)

### Commentaire :
ici la valeur optimale de alpha de la regression ridge est : alpha = 1 

# 4.3. Régression linéaire pénalisée : Lasso

### Reponses aux questions

la régression lasso est un type de régression linéaire qui permet de réduire les limites du modèle. Les valeurs des données se réduisent au centre ou à la moyenne pour éviter de surcharger les données.

## Application de la régression Lasso

In [ ]:
lasso = linear_model.Lasso()

#definition des valeur de alpha
params_Lasso = {'alpha': [0.1, 1, 10] , "fit_intercept": [True, False]}


#Recherche de meilleur parametre
Lasso_GS = GridSearchCV(lasso,
                        param_grid=params_Lasso,
                       )
#Apprentissage du model
lasso = Lasso_GS.fit(X_av, Y_av)

scores = Lasso_GS.cv_results_["mean_test_score"]
scores_std = Lasso_GS.cv_results_["std_test_score"]

print("Résultats de la validation Croisé (Lasso) sur l'ensemble des paramètres:")   
print("Meilleurs paramètres alpha du Lasso :")
print(Lasso_GS.best_params_)

### Commentaire :
ici la valeur optimale de alpha de la regression Lasso est : alpha = 0.1 